### Prompt Engineering OUTPATIENT HEALTHCARE + VERIFICATION ###
Add another output to confirm if the prediction can be verified from the information given

In [ ]:
import os
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import MentalHealth, OutpatientServices
from llmt.llmtools import process_prompt
from llmt.openai import OpenAI, create_messages
from llmt.performance import Performance

In [ ]:
# Parameters
model = 'gpt-4o'
temperature = 0

# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
test_file_name = 'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df = pd.read_parquet(test_file)
# Filter the labeled data
df = df.loc[df['dset'] == 'train'].\
                reset_index(drop=True).\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})
display(df.head())
print(df.shape)

### Prompt development ###

In [ ]:
# Pick an example of a company that provides mental health services
mental_health = 1
outpatient = 1
id_list = list(df.loc[(df['mental_health'] == mental_health) & (df['outpatient'] == outpatient), 'id'].unique())
print(f'Found {len(id_list)} company ids')

# Pick a company ID
idx = 5
company_id = id_list[idx]
# company_id = '135432-64'

ser_id = df.loc[df['id'] == company_id].iloc[0]
display(ser_id)
name = ser_id['name']
description = ser_id['description']
print()
print(name)
print(description)

In [ ]:
variable = 'outpatient'
prompt_version = 1
prompt_name = f'{variable}_system_{str(prompt_version).zfill(2)}'
system_prompt = Prompt().load(prompt_name=prompt_name)

# For the user prompt, we want to include the name of the business and the description
def create_user_prompt(name: str, description: str, variable: str):
    user_prompt = f"""
        The organization {name} is described as: {description} 
        Does this organization provide {variable} healthcare services?
        """
    return process_prompt(user_prompt)

user_prompt = create_user_prompt(name=name, description=description, variable=variable)
messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)

In [ ]:
print(system_prompt)
print()
print(user_prompt)

In [ ]:
# Send prompt to model
model = 'gpt-4o'
temperature = 0
response_format = OutpatientServices
client = OpenAI().create_client()

output = OpenAI().send_messages(messages=messages, 
                                model=model, 
                                response_format=response_format,
                                temperature=0,
                                client=client)
print(output)
# Replace the boolean with binary outcome prediction
key_list = ['pred_op', 'verified_op']
output.update({key: 1 if output.get(key) == True else 0 for key in key_list})
print(output)

In [ ]:
# Run the prompt on all data
company_id_list = sorted(list(df['id'].unique()))
results_df_list = []

for c, company_id in enumerate(company_id_list):
    if (c + 1) % 20 == 0:
        print(f'Sending description {c + 1} / {len(company_id_list)} to the model')
    df_id = df.loc[df['id'] == company_id]
    user_prompt_id = create_user_prompt(name=df_id['name'].values[0], 
                                        description=df_id['description'].values[0],
                                        variable=variable)
    messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt_id)
    output = OpenAI().send_messages(messages=messages,
                                    model=model,
                                    temperature=temperature,
                                    response_format=response_format,
                                    client=client)
    
   # Replace the boolean with binary outcome prediction
    key_list = ['pred_op', 'verified_op']
    output.update({key: 1 if output.get(key) == True else 0 for key in key_list})
    results_df_list.append(df_id.assign(**output))

results_df = pd.concat(results_df_list, axis=0, ignore_index=True)
# Save the results
results_file_name = f'{variable}_{str(prompt_version).zfill(2)}_results.parquet'
results_file = os.path.join(data_dir, results_file_name)
results_df.to_parquet(results_file)
print(results_file)

In [ ]:
display(Performance(data=results_df).\
        binary_performance(true_col='outpatient', pred_col='pred_op'))